In [2]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-fd7c  GCE       6                                       RUNNING  us-central1-a


# Imports & Setup

In [3]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [4]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 15 13:20 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [6]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [7]:
spark

In [17]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'bucket1_314978370'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and b.name.startswith('multi'):
        paths.append(full_path+b.name)

print(len(paths))




['gs://bucket1_314978370/multistream10_preprocessed.parquet', 'gs://bucket1_314978370/multistream11_part2_preprocessed.parquet', 'gs://bucket1_314978370/multistream11_preprocessed.parquet', 'gs://bucket1_314978370/multistream12_part2_preprocessed.parquet', 'gs://bucket1_314978370/multistream12_preprocessed.parquet', 'gs://bucket1_314978370/multistream13_part2_preprocessed.parquet', 'gs://bucket1_314978370/multistream13_preprocessed.parquet', 'gs://bucket1_314978370/multistream14_part2_preprocessed.parquet', 'gs://bucket1_314978370/multistream14_preprocessed.parquet', 'gs://bucket1_314978370/multistream15_part2_preprocessed.parquet', 'gs://bucket1_314978370/multistream15_part3_preprocessed.parquet', 'gs://bucket1_314978370/multistream15_preprocessed.parquet', 'gs://bucket1_314978370/multistream16_part2_preprocessed.parquet', 'gs://bucket1_314978370/multistream16_part3_preprocessed.parquet', 'gs://bucket1_314978370/multistream16_preprocessed.parquet', 'gs://bucket1_314978370/multistream1

In [11]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [12]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [13]:
from inverted_index_gcp import InvertedIndex


In [14]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords]
  word_count = Counter(tokens)
  result = [(token,(id, tf)) for token, tf in word_count.items()]
  return result


def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])
  return sorted_pl


def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  results = postings.map(lambda x: (x[0], len(x[1])))

  return results


def partition_postings_and_write(postings, folder_name):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''

  pp = postings.map(lambda x: (token2bucket_id(x[0]), x))
  pp = pp.groupByKey()
  pl = pp.map(lambda x: InvertedIndex.write_a_posting_list(x,bucket_name, folder_name))

  return pl

# Building an inverted index for body


In [18]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd

In [ ]:
DL = doc_text_pairs.map(lambda x: ((x[1]), len(x[0])))
DL_df = sqlContext.createDataFrame(DL)
df = DL_df.toPandas()
df.to_csv('DL.csv')

In [19]:
folder = "body"

# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered, folder).collect()
index_const_time = time() - t_start
print(index_const_time)


1318.2451479434967


In [45]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='body'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)


In [46]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'body_index')
# upload to gs
index_src = "body_index.pkl"
index_dst = f'gs://{bucket_name}/{folder}/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://body_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 15.4 MiB/ 15.4 MiB]                                                
Operation completed over 1 objects/15.4 MiB.                                     


In [47]:
!gsutil ls -lh $index_dst

  15.4 MiB  2023-01-15T16:26:13Z  gs://bucket1_314978370/anchor/body_index.pkl
TOTAL: 1 objects, 16145159 bytes (15.4 MiB)


# Building an inverted index for title


In [23]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("title", "id").rdd

In [ ]:
folder = "title2"
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered, folder).collect()
index_const_time = time() - t_start
print(index_const_time)


40.30591869354248


In [48]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title2'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)


In [49]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'title_index')
# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/{folder}/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://title_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][  3.8 MiB/  3.8 MiB]                                                
Operation completed over 1 objects/3.8 MiB.                                      


In [50]:
!gsutil ls -lh $index_dst



  3.79 MiB  2023-01-15T16:26:50Z  gs://bucket1_314978370/anchor/title_index.pkl
TOTAL: 1 objects, 3977331 bytes (3.79 MiB)


# Building an inverted index for anchor text


In [39]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("anchor_text").rdd

In [40]:
doc_text_pairs = doc_text_pairs.flatMap(lambda x: x)
doc_text_pairs = doc_text_pairs.flatMap(lambda x: x)

In [41]:

folder = "anchor"
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[1], x[0]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered, folder).collect()
index_const_time = time() - t_start
print(index_const_time)


802.5007157325745


In [51]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='anchor'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)


In [52]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'anchor_index')
# upload to gs
index_src = "anchor_index.pkl"
index_dst = f'gs://{bucket_name}/{folder}/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://anchor_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][  9.5 MiB/  9.5 MiB]                                                
Operation completed over 1 objects/9.5 MiB.                                      


In [53]:
!gsutil ls -lh $index_dst

  9.46 MiB  2023-01-15T16:27:22Z  gs://bucket1_314978370/anchor/anchor_index.pkl
TOTAL: 1 objects, 9917184 bytes (9.46 MiB)


# Create PageRank file for corpus


In [64]:
def make_graph(data):
  dest = dict.fromkeys(list(map(lambda x: x[0], data[1])))
  return list(map(lambda x: (data[0],x), dest))


def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
  '''
  edges = pages.flatMap(make_graph)
  vertices = edges.flatMap(lambda x: x).distinct().map(lambda x: [x])

  return edges, vertices

In [65]:
t_start = time()
pages_links =spark.read.parquet(*paths).select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()
print(pr_time)



[(4045403, 1176764), (4045403, 91416), (4045403, 136747), (4045403, 23814944), (4045403, 185235), (4045403, 32090), (4045403, 29810), (4045403, 32927), (4045403, 325329), (4045403, 136815)]


+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750|  9913.72878216077|
|  10568| 5385.349263642033|
|  32927| 5282.081575765272|
|  30680|  5128.23370960412|
|5843419|4957.5676862638675|
|  68253| 4769.278265355158|
|  31717| 4486.350180548307|
|  11867| 4146.414650912769|
|  14533| 3996.466440885499|
| 645042|3531.6270898037396|
|  17867|3246.0983906041397|
|5042916| 2991.945739166177|
|4689264|2982.3248830417456|
|  14532|2934.7468292031695|
|  25391|2903.5462235133987|
|   5405| 2891.416329154635|
|4764461| 2834.366987332659|
|  15573|2783.8651181588366|
|   9316| 2782.039646413768|
|8569916|2775.2861918400154|
+-------+------------------+
only showing top 20 rows

725.9859364032745


In [66]:
df = pr.toPandas()
df.to_csv('PR.csv')

In [67]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

14.28 GiB    gs://wikidata_preprocessed


In [69]:
# size of index data
index_dst = f'gs://{bucket_name}/body/'
!gsutil du -sh "$index_dst"

5.92 GiB     gs://bucket1_314978370/body


# Create PageView file for corpus

In [ ]:
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path)
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB)
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly
# total number of page views (5). Then, remove lines with article id or page
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)
# read in the counter
# with open(pv_clean, 'rb') as f:
#   wid2pv = pickle.loads(f.read())

--2023-01-15 17:23:22--  https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘pageviews-202108-user.bz2’ not modified on server. Omitting download.

